In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv("train.csv")

In [2]:
X = data.drop('label', axis=1).values.T        
Y = data['label'].values.reshape(1, -1)        

X = X / 255.0


m = X.shape[1]
split = int(0.8 * m)  

X_train = X[:, :split]
Y_train = Y[:, :split]
X_test = X[:, split:]
Y_test = Y[:, split:]


In [3]:
def one_hot(Y):
    one_hot_Y = np.zeros((10, Y.size))
    one_hot_Y[Y, np.arange(Y.size)] = 1
    return one_hot_Y

Y_train_oh = one_hot(Y_train.flatten())
Y_test_oh = one_hot(Y_test.flatten())


In [4]:
def init_params(input_size, hidden_size, output_size):
    W1 = np.random.randn(hidden_size, input_size) * np.sqrt(2 / input_size)
    b1 = np.zeros((hidden_size, 1))
    W2 = np.random.randn(output_size, hidden_size) * np.sqrt(2 / hidden_size)
    b2 = np.zeros((output_size, 1   ))
    return W1, b1, W2, b2


In [5]:
def relu(Z):
    return np.maximum(0, Z)

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True))  
    return expZ / expZ.sum(axis=0, keepdims=True)

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1 @ X + b1
    A1 = relu(Z1)
    Z2 = W2 @ A1 + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2


In [6]:
def relu_derivative(Z):
    return Z > 0

def backward_prop(Z1, A1, A2, W2, X, Y):
    m = X.shape[1]
    dZ2 = A2 - Y
    dW2 = (1/m) * dZ2 @ A1.T
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
    
    dZ1 = (W2.T @ dZ2) * relu_derivative(Z1)
    dW1 = (1/m) * dZ1 @ X.T
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)
    
    return dW1, db1, dW2, db2


In [7]:
def update(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 -= alpha * dW1
    b1 -= alpha * db1
    W2 -= alpha * dW2
    b2 -= alpha * db2
    return W1, b1, W2, b2


In [8]:
def train(X, Y, hidden_size=128, alpha=0.1, iterations=1000):
    input_size = X.shape[0]
    output_size = 10
    W1, b1, W2, b2 = init_params(input_size, hidden_size, output_size)

    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, A2, W2, X, Y)
        W1, b1, W2, b2 = update(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)

        if i % 100 == 0:
            acc = get_accuracy(get_predictions(A2), np.argmax(Y, axis=0))
            print(f"Iteration {i}: Accuracy = {acc:.4f}")
    return W1, b1, W2, b2


In [9]:
def get_predictions(A2):
    return np.argmax(A2, axis=0)

def get_accuracy(predictions, Y):
    return np.mean(predictions == Y.flatten())

def evaluate_model(W1, b1, W2, b2, X, Y):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    preds = get_predictions(A2)
    acc = get_accuracy(get_predictions(A2), Y)
    print("Accuracy:", acc)
    return acc


In [10]:
W1, b1, W2, b2 = train(X_train, Y_train_oh, hidden_size=128, alpha=0.1, iterations=1000)
print("Train accuracy:")
evaluate_model(W1, b1, W2, b2, X_train, Y_train)
print("Test accuracy:")
evaluate_model(W1, b1, W2, b2, X_test, Y_test)


Iteration 0: Accuracy = 0.0662
Iteration 100: Accuracy = 0.8834
Iteration 200: Accuracy = 0.9038
Iteration 300: Accuracy = 0.9130
Iteration 400: Accuracy = 0.9202
Iteration 500: Accuracy = 0.9258
Iteration 600: Accuracy = 0.9308
Iteration 700: Accuracy = 0.9353
Iteration 800: Accuracy = 0.9384
Iteration 900: Accuracy = 0.9416
Train accuracy:
Accuracy: 0.9451488095238095
Test accuracy:
Accuracy: 0.9420238095238095


np.float64(0.9420238095238095)